<a href="https://colab.research.google.com/github/Minsoo1036/Algorithm-Study/blob/main/rpy2_practice_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade rpy2==3.4.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 KB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rpy2: filename=rpy2-3.4.5-cp39-cp39-linux_x86_64.whl size=297796 sha256=a5b5bcc6495a99f8d3f888eddb95351856ab792075196584bef404fad2da48da
  Stored in directory: /root/.cache/pip/wheels/36/d1/63/882c1f63d21bcf817b7db960b9536a747d4258daeaace0edd4
Successfully built rpy2
  Attempting uninstall: rpy2
    Found existing installation: rpy2 3.5.5
    Uninstalling rpy2-3.5.5:
      Successfully uninstalled rpy2-3.5.5


In [2]:
import pandas as pd
import numpy as np

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
import rpy2.robjects.packages as rpackages
from rpy2.robjects import numpy2ri
from rpy2.robjects.vectors import StrVector

from statsmodels.stats.anova import anova_lm 
import statsmodels.formula.api as smf

required_packages = ['base', 'forecast', 'CondIndTests','devtools'] # list of required R packages 
if all(rpackages.isinstalled(x) for x in required_packages):
    check_packages = True # True if packages are already installed 
else:
   check_packages = False # False if packages are not installed 
if check_packages == False: # Not installed? Then install.
    utils = rpackages.importr('utils')
    utils.chooseCRANmirror(ind=1)
    packages_to_install = [x for x in required_packages if not rpackages.isinstalled(x)]
    if len(packages_to_install) > 0:
        utils.install_packages(StrVector(packages_to_install))
    check_packages = True 

r = ro.r
base = importr('base')
forecast = importr('forecast')
graphics = importr('graphics')
grdevices = importr('grDevices')
condindtests = importr('CondIndTests')
devtools = importr('devtools')

(as ‘lib’ is unspecified)




































































































































































	‘/tmp/RtmpJ5bnk3/downloaded_packages’



In [3]:
required_packages = ['CausalImpact'] # list of required R packages 
if all(rpackages.isinstalled(x) for x in required_packages):
    check_packages = True # True if packages are already installed 
else:
   check_packages = False # False if packages are not installed 
if check_packages == False: # Not installed? Then install.
    utils = rpackages.importr('utils')
    utils.chooseCRANmirror(ind=1)
    packages_to_install = [x for x in required_packages if not rpackages.isinstalled(x)]
    if len(packages_to_install) > 0:
        utils.install_packages(StrVector(packages_to_install))
    check_packages = True 

CausalImpact = importr('CausalImpact')

(as ‘lib’ is unspecified)





























	‘/tmp/RtmpJ5bnk3/downloaded_packages’



In [50]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima_process import ArmaProcess
#from causalimpact import CausalImpact


np.random.seed(12345)
ar = np.r_[1, 0.9]
ma = np.array([1])
arma_process = ArmaProcess(ar, ma)
X = 100 + arma_process.generate_sample(nsample=100)
y = 1.2 * X + np.random.normal(size=100)
y[70:] += 5

data = pd.DataFrame({'y': y, 'X': X}, columns=['y', 'X'])
pre_period = [1, 70]
post_period = [71, 101]


In [51]:
data

,y,X
0,118.188694,99.795292
1,120.233276,100.663180
2,118.627775,98.883699
3,119.609722,100.448941
4,121.391508,101.561734
...,...,...
95,131.838436,104.939544
96,118.786121,95.672520
97,127.362025,103.146201
98,123.600675,97.753389


In [48]:
#data.to_csv("train_bsts.csv")

In [52]:
with localconverter(ro.default_converter + pandas2ri.converter):
    r_from_pd_explorer = ro.conversion.py2rpy(data)

In [53]:
r_from_pd_explorer

y,X
...,...


In [54]:
time_points = r.seq(1,101,1)
r_from_pd_explorer = r.zoo(r_from_pd_explorer,time_points)

In [55]:
with localconverter(ro.default_converter + pandas2ri.converter):
    r_from_pd_explorer1 = ro.conversion.py2rpy(pre_period)

with localconverter(ro.default_converter + pandas2ri.converter):
    r_from_pd_explorer2 = ro.conversion.py2rpy(post_period)

In [56]:
r.unlist(r_from_pd_explorer1)

1,70


In [57]:
pre = r.unlist(r_from_pd_explorer1)

In [58]:
post = r.unlist(r_from_pd_explorer2)

In [59]:
tm = pd.DataFrame({"niter":[5000], "nseasons":[12], "prior.level.sd":[0.05]})


In [60]:
tm

,niter,nseasons,prior.level.sd
0,5000,12,0.05


In [61]:
with localconverter(ro.default_converter + pandas2ri.converter):
    tm2 = ro.conversion.py2rpy(tm)

In [62]:
tm2

niter,nseasons,prior.level.sd
...,...,...


In [63]:
fit = r.CausalImpact(r_from_pd_explorer,pre,post,tm2)

In [64]:
fit

series,[RTYPES.REALSXP]
summary,[RTYPES.VECSXP]
report,[RTYPES.STRSXP]
model,[RTYPES.VECSXP]


In [65]:
r.summary(fit)

Posterior inference {CausalImpact}

                         Average        Cumulative   
Actual                   125            3875         
Prediction (s.d.)        121 (0.68)     3742 (21.18) 
95% CI                   [119, 122]     [3702, 3783] 
                                                     
Absolute effect (s.d.)   4.3 (0.68)     133.3 (21.18)
95% CI                   [3, 5.6]       [92, 173.6]  
                                                     
Relative effect (s.d.)   3.6% (0.59%)   3.6% (0.59%) 
95% CI                   [2.4%, 4.7%]   [2.4%, 4.7%] 

Posterior tail-area probability p:   2e-04
Posterior prob. of a causal effect:  99.97997%

For more details, type: summary(impact, "report")



<rpy2.rinterface_lib.sexp.NULLType object at 0x7efc4807c700> [RTYPES.NILSXP]

In [66]:
import numpy as np
tem2 = np.array(fit[0])

In [67]:
tem2.shape

(101, 14)

In [68]:
pd.DataFrame(tem2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,118.188694,118.188694,119.690044,117.115512,122.155170,118.188694,118.188694,118.188694,-1.501351,-3.966477,1.073181,0.000000,0.000000,0.000000
1,120.233276,238.421970,120.253690,117.668885,122.856868,238.421970,238.421970,238.421970,-0.020414,-2.623592,2.564392,0.000000,0.000000,0.000000
2,118.627775,357.049744,117.905831,115.403185,120.564312,357.049744,357.049744,357.049744,0.721944,-1.936537,3.224590,0.000000,0.000000,0.000000
3,119.609722,476.659467,120.598626,118.032279,123.191618,476.659467,476.659467,476.659467,-0.988903,-3.581896,1.577443,0.000000,0.000000,0.000000
4,121.391508,598.050975,121.295878,118.759188,123.875477,598.050975,598.050975,598.050975,0.095631,-2.483969,2.632320,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,118.786121,11770.976293,115.504244,112.418685,118.560202,11650.592306,11616.153359,11686.004155,3.281877,0.225919,6.367437,120.383986,84.972137,154.822934
97,127.362025,11898.338317,123.869829,120.813904,126.999702,11774.462136,11738.226388,11811.445692,3.492196,0.362323,6.548121,123.876182,86.892625,160.111930
98,123.600675,12021.938992,117.176081,114.194032,120.216837,11891.638217,11853.920882,11930.146132,6.424594,3.383838,9.406643,130.300776,91.792860,168.018110
99,127.861827,12149.800819,123.305537,120.253564,126.314927,12014.943753,11975.853838,12055.030081,4.556290,1.546900,7.608263,134.857066,94.770738,173.946981


In [ ]:
#pd.DataFrame(tem2).to_csv("causal.csv")